# Renewable energy prediction
Helpful notebook for the prediction of energy from renewable resources.

Author: David Salac <http://www.github.com/david-salac>

## Solar photovoltaic power plants prediction
There are many ways how to predict the production of photovoltaic installations. The represented one uses the PVLIB library (that is publicly available under a commercially friendly license).

### What are the inputs
The list of inputs follows. The vendor provides some of them, some of them are dependant on location of installation, and remaining are dependant on the time of prediction.

#### Environmental variables
Weather conditions and current time define them.

1. _E<sub>GHI</sub>_: acronym of the Global Horizontal Irradiance, units: W⋅m<sup>-2</sup>

2. _T_: temperature, units: &deg;C

3. _R_: ground albedo (reflectance of the ground around), units: unitless (value from 0 to 1)

4. _t_: calendar time with precision up to seconds, typically in UTC timezone.

#### Installation dependant parameters
Positional (depends on spatial solution):

1. _A<sub>t</sub>_: tilt angle, units: degree (&deg;)

2. _A<sub>a</sub>_: azimuth angle, units: degree (&deg;)

3. _P<sub>lon, lat</sub>_: longitude and latitude of the location, units: degree (&deg;)

Defined by vendor:

1. _G<sub>s</sub>_: linear coefficient of efficiency decrease when 1&deg;C temperature increase happens, units: unitless

2. _T<sub>opt</sub>_: operational temerature of installation, units: &deg;C

3. _E<sub>std</sub>_: standard irradiance, units: W⋅m<sup>-2</sup>

4. _T<sub>std</sub>_: standard temperature, units: &deg;C

5. _P_: maximal possible power to be produced (capacity), units: W

### Prediction logic
How the prediction works is presented in the following example:

In [ ]:
# =============================================================================
from typing import Tuple
from datetime import datetime

import pvlib


def solar(
    E_ghi: float, T: float, R: float, t: datetime, *,
    A_t: float, A_a: float, P_lon_lat: Tuple[float, float],
    G_s: float, T_opt: float, E_std: float, T_std: float, P: float
) -> float:
    """
    Predict the photovoltaic power production.

    :param E_ghi: global horizontal irradiance
    :param T: temperature
    :param R: ground albedo (reflectance of the ground around)
    :param t: calendar time with precision up to seconds, in UTC
    :param A_t: tilt angle
    :param A_a: azimuth angle
    :param P_lon_lat: longitude and latitude of the location
    :param G_s: linear coefficient of efficiency decrease
    :param T_opt: operational temerature of installation
    :param E_std: standard irradiance
    :param T_std: standard temperature
    :param P: capacity of installation

    :return: The power prediction of installation in Watts.
    """
    # Compute the Solar Zenith Angle and Solar Azimuth Angle
    sza_saa = pvlib.solarposition.get_solarposition(t,
                                                    P_lon_lat[1], P_lon_lat[0])
    # Extract SZA and SAA
    A_sza, A_saa = sza_saa['zenith'], sza_saa['azimuth']
    # Compute DHI and DNI (Direct Normal Irradiance and
    # Diffuse Horizontal Irradiance) using Erbs model
    dhi_dni = pvlib.irradiance.erbs(E_ghi, A_sza, t)
    # Extract DHI and DNI
    E_dhi, E_dni = dhi_dni['dhi'], dhi_dni['dni']
    # Computes the total irradiance
    irr = pvlib.irradiance.get_total_irradiance(A_t, A_a, A_sza, A_saa,
                                                E_dni, E_ghi, E_dhi, albedo=R)
    # Extract total irradiance
    E_irr = irr['poa_global'].to_numpy().squeeze()

    # Normalized temperature
    T_n = T + E_irr * (T_opt - 20.0) / 800.0

    # Predict the power:
    return (E_irr / E_std) * (1.0 + G_s) * (T_n - T_std) * P


# EXAMPLE
power = solar(E_ghi=800.0, T=23.5, R=0.25, t=datetime(2020, 5, 12, 10, 30),
              A_t=30.0, A_a=30.0, P_lon_lat=(-0.12795, 51.50774),
              G_s=-0.004, T_opt=47.0, E_std=1000.0, T_std=43.0, P=1000.0)
